<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Earth Engine API
!pip install earthengine-api

# Authenticate and initialize GEE
import ee
from google.colab import auth
auth.authenticate_user()

# Initialize Earth Engine
ee.Initialize()

In [ ]:
# Define a region (replace with Tainan City geometry)
geometry = ee.Geometry.Polygon([
  [[120.0093, 22.7373], [120.3001, 22.7373],
   [120.3001, 23.3308], [120.0093, 23.3308],
   [120.0093, 22.7373]]
])

In [ ]:

var startDate = ee.Date('2023-01-01');
var endDate = startDate.advance(1, 'year');

var exportResolutionMeters = 30;

var waterbodySizeMeters = 2000;
var islandSizeMeters = 1000;

var collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
  .filter(ee.Filter.date(startDate, endDate))
  .filter(ee.Filter.lt('CLOUD_COVER', 30))
  .filter(ee.Filter.bounds(geometry));

var collection = collection.select(
  ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
  ['blue',  'green', 'red',   'nir',   'swir1', 'swir2']
);

var composite = collection.median().clip(geometry);

var rgbVis = {
  bands: ['red', 'green', 'blue'],
  min: 1,
  max: 30000
};

Map.addLayer(composite, rgbVis, 'Satellite Image')

# Gao
var ndwi = composite.normalizedDifference(['nir', 'swir1']).rename('ndwi');
Map.addLayer(ndwi, {min:-1, max:1, palette: ['white', 'black']}, 'NDWI Image')

In [1]:
var detectWater = function(image) {
 # Otsu Thresholding
  var thresholding = require('users/gena/packages:thresholding');
  var scale = 100;
  var bounds = geometry;
  var cannyThreshold = 0.7;
  var cannySigma = 1;
  var minValue =  0.0045;
  var th = thresholding.computeThresholdUsingOtsu(
    ndwi, scale, bounds, cannyThreshold, cannySigma, minValue);

  var water = ndwi.gt(th).selfMask().rename('water');

  return water;
};

var water = detectWater(composite);
Map.addLayer(water, {palette: ['white', 'black']}, 'NDWI Binarization')

var coastline = water.reduceToVectors({
  geometryType: 'polygon',
  reducer: ee.Reducer.countEvery(),
  scale: 10,
  geometry: geometry
});

Map.addLayer(coastline, {color: 'blue'}, 'Land-Water Boundary (Vector)');
Map.centerObject(geometry, 10);


# This function takes vectorized polygons and
# extracts a polyline
var simplifyAndExtractCoastline = function(coastline){
  # Simplify vectors
  var processedVectors = coastline.map(function(f) {
    var coords = f.geometry()
      .simplify({maxError: exportResolutionMeters})
      .coordinates();

    # Buffer the geometry by a pixel to avoid rasterizing
    # the boundary polygon
    var bufferDistance = ee.Number(
      exportResolutionMeters).multiply(-1);
    return f
      .setGeometry(
        ee.Geometry.MultiLineString(coords)
          .intersection(geometry.buffer(bufferDistance)));
  });
  return processedVectors;
};

var coastlineVector = simplifyAndExtractCoastline(coastline);
Map.addLayer(coastlineVector, {color: 'red'},
  'Coastline (Vector)');
Map.centerObject(geometry, 12);

SyntaxError: invalid syntax (<ipython-input-1-d4cb9f052d8c>, line 1)

In [ ]:
def detect_change(mndwi_current, mndwi_next, threshold=0.2):
    # Calculate the difference between consecutive years
    mndwi_diff = mndwi_next.subtract(mndwi_current).abs()

    # Classify change based on the threshold
    land_to_water = mndwi_diff.gt(threshold).And(mndwi_next.lt(mndwi_current))
    water_to_land = mndwi_diff.gt(threshold).And(mndwi_next.gt(mndwi_current))

    # Generate a change map: 1 = land to water (red), -1 = water to land (blue), 0 = no change (black)
    change_map = land_to_water.multiply(1).add(water_to_land.multiply(-1))
    return change_map

In [ ]:
years = list(range(2000, 2023))

# Store the yearly MNDWI maps
mndwi_maps = [composite for year in years]

# Detect changes between consecutive years
change_maps = []
for i in range(len(mndwi_maps) - 1):
    change = detect_change(mndwi_maps[i], mndwi_maps[i+1])
    change_maps.append(change)

In [ ]:
# Example visualization parameters
viz_params = {
    'min': -1,
    'max': 1,
    'palette': ['0000FF', '000000', 'FF0000']  # Blue, Black, Red
}

# Visualize a change map (e.g., between 2000 and 2001)
import folium
map_ = folium.Map(location=[23.0, 120.25], zoom_start=10)
map_.add_ee_layer(change_maps[0], viz_params, 'Change Map 2000-2001')
map_

In [ ]:
export_task = ee.batch.Export.image.toDrive(**{
    'image': change_maps[0],
    'description': 'MNDWI_change_2000_2001',
    'scale': 30,
    'region': tainan,
    'maxPixels': 1e9
})
export_task.start()